In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
import torch
# from datasets import Dataset

In [2]:
cache_dir = '/home/zungwooker/Debiasing/pretrained/bart'

In [3]:
model = AutoModelForSequenceClassification.from_pretrained("facebook/bart-large-mnli", cache_dir=cache_dir)
device = torch.device(f'cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-mnli", cache_dir=cache_dir)

In [4]:
classifier = pipeline("zero-shot-classification", model=model, tokenizer=tokenizer, device=device)

In [5]:
sequence_to_classify = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
candidate_labels = ['sky','park','tree','sit','field','cat','cat','person','water','street'	]
input_pairs = zip(sequence_to_classify, candidate_labels)
# for p in input_pairs:
#     res = classifier(p[0], p[1], multi_label=True)
#     print(res['sequence'])
#     print(res['labels'])
#     print(res['scores'])
res = classifier(sequence_to_classify, candidate_labels, multi_label=True)

/home/zungwooker/anaconda3/envs/gene/lib/python3.9/site-packages/torch/utils/data/dataloader.py:693: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7f32649c68e0> was reported to be 10 (when accessing len(dataloader)), but 11 samples have been fetched. 
  warnings.warn(warn_msg)
/home/zungwooker/anaconda3/envs/gene/lib/python3.9/site-packages/torch/utils/data/dataloader.py:693: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7f32649c68e0> was reported to be 10 (when accessing len(dataloader)), but 12 samples have been fetched. 
  warnings.warn(warn_msg)
/home/zungwooker/anaconda3/envs/gene/lib/python3.9/site-packages/torch/utils/data/dataloader.py:693: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7f32649c68e0> was reported to be 10 (when accessing len(dataloader)), but 13 samples have been fetched. 
  warnings.w

In [12]:
res[0]['labels'][1]
res[0]['scores'][1]

0.2504567503929138

In [28]:
a = {0: 'a', 1: 'b'}
[value for value in a.values()]

['a', 'b']

In [20]:
filtered_tags_all = []
for i in range(10):
    filtered_tags_all += [label for score, label in zip(res[i]['scores'], res[i]['labels']) if score >= 0.93]
    
filtered_tags_all = set(filtered_tags_all)

In [26]:
a = set([1,2,3])
b = set([3,4,5])
print(list(a-b))

[1, 2]


In [21]:
filtered_tags_all

{'cat', 'sky'}

In [22]:
sequence_to_classify = set(['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']) - filtered_tags_all

In [23]:
sequence_to_classify

{'airplane',
 'automobile',
 'bird',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck'}

In [7]:
filtered_scores_labels = [(score, label) for score, label in zip(res['scores'], res['labels']) if score <= 0.99]
res['scores'], res['labels'] = zip(*filtered_scores_labels) if filtered_scores_labels else ([], [])

TypeError: list indices must be integers or slices, not str

In [ ]:
print(res['sequence'])
print(res['labels'])
print(res['scores'])

A photo of young
('girl', 'woman', 'person', 'face', 'pose', 'blue', 'love', 'hair', 'dress', 'mouse', 'laptop', 'costume', 'code', 'computer')
(0.9812822341918945, 0.8993655443191528, 0.8792117238044739, 0.6589139699935913, 0.4070006310939789, 0.17151901125907898, 0.11285580694675446, 0.03831982612609863, 0.02029099501669407, 0.006456996314227581, 0.0019525282550603151, 0.0011518665123730898, 0.0011386561673134565, 0.0004380836326163262)
